In [1]:
import os
from tqdm import tqdm
import torch
import cv2
import numpy as np
from PIL import Image
from torchvision import transforms
import csv
from time import time

### Get frames

In [2]:
data_dir = r"D:\VGAF_dataset"
print(data_dir)

D:\VGAF_dataset


In [11]:
def process_dir(dirname):
    resdir = data_dir + "/" + dirname + '_frames'
    d = os.path.normpath(os.path.join(data_dir, dirname))
    for filename in tqdm(os.listdir(d)):
        '''if filename.lower().endswith('ini'):
            continue'''
        if os.path.isdir(os.path.normpath(os.path.join(d,filename))):
            videofile=None
            for fn in os.listdir(os.path.normpath(os.path.join(d,filename))):
                '''if fn.lower().endswith('ini'):
                    continue'''
                videofile=fn
            if videofile is None:
                continue
            filename=os.path.normpath(os.path.join(filename,videofile))
        fn, ext = os.path.splitext(os.path.basename(filename))
        outdir=os.path.normpath(os.path.join(resdir, fn))
        if not os.path.exists(outdir):
            os.makedirs(outdir)
        command = "ffmpeg -r 1 -i "+os.path.join(d,filename) + " -r 1 "+outdir+"/%05d.png"
        command = os.path.normpath(command)
        # print(command)
        os.system(command=command)

In [12]:
process_dir('Train')

100%|██████████| 2661/2661 [10:04:21<00:00, 13.63s/it]   


In [13]:
process_dir('Val')

100%|██████████| 766/766 [1:27:44<00:00,  6.87s/it]


### Detect faces in frames

In [2]:
print(f"Torch: {torch.__version__}")
# device = 'cuda:0'
device = 'cpu'
use_cuda = torch.cuda.is_available()
print(use_cuda)

Torch: 1.9.1
True


In [4]:
from facenet_pytorch import MTCNN # pretrained model for image recognition
mtcnn = MTCNN(keep_all=True, min_face_size=40, device=device)

In [5]:
from facial_analysis import FacialImageProcessing
imgProcessing=FacialImageProcessing(False,minsize=64)

In [8]:
scale=1
def save_faces(source_path,save_path):
    if not os.path.exists(save_path):
        os.mkdir(os.path.normpath(save_path))
    for folder in tqdm(os.listdir(source_path)):
        if not os.path.exists(os.path.join(save_path, folder)):
            os.mkdir(os.path.normpath(os.path.join(save_path, folder)))
    
        for image in os.listdir(os.path.join(source_path, folder)):
            filename = os.path.join(source_path, folder, image)
            # print(filename)
            frame_bgr = cv2.imread(filename)
            frame = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
            #frame=cv2.resize(frame, (0,0), fx=1/scale, fy=1/scale) 
            bounding_boxes, _ = imgProcessing.detect_faces(frame)

            if len(bounding_boxes)!=0:
                root,ext=os.path.splitext(image)
                faces_folder=os.path.join(save_path, folder, root) 
                if not os.path.exists(faces_folder):
                    os.mkdir(faces_folder)
                for i,bounding_box in enumerate(bounding_boxes):
                    outfile=os.path.join(faces_folder, str(i)+ext)
                    if not os.path.exists(outfile):
                        bounding_box*=scale
                        b=[max(0,int(bi)) for bi in bounding_box]
                        x1,y1,x2,y2=b[0:4]
                        face_img=frame_bgr[y1:y2,x1:x2,:]

                        if np.prod(face_img.shape)==0:
                            print('Empty face ',b,' found for ',filename)
                            continue
                        #face_img=cv2.resize(face_img,INPUT_SIZE)
                        cv2.imwrite(outfile, face_img) 
        
        
save_faces(os.path.join(data_dir,'Val_frames'),os.path.join(data_dir,'Val_faces'))
save_faces(os.path.join(data_dir,'Train_frames'),os.path.join(data_dir,'Train_faces'))

100%|██████████| 2661/2661 [19:33:10<00:00, 26.45s/it]   
